In [18]:
from plotnine import ggplot, geom_point, geom_line, aes, geom_label 
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
import pandas as pd

@interact(n_pop=200000000, n_days=300, print_daily=False, 
          beta_daily_inf_rate=widgets.FloatSlider(min=0.0, max=1, step=.01, value=.3),
          n_patient_zero=10, 
          mean_days_infectuous=widgets.FloatSlider(min=0.0, max=30, step=1, value=10),
          death_prob=widgets.FloatSlider(min=0.0, max=.2, step=.01, value=.02),
          mean_days_to_death=10)
def sirtd_exact_sim(n_pop, n_days, print_daily, beta_daily_inf_rate, n_patient_zero, 
                    mean_days_infectuous, death_prob, mean_days_to_death):
    S = [n_pop - n_patient_zero]
    I = [n_patient_zero]
    R = [0]
    T = [0]
    D = [0]
    tweets = [0]
    for today in range(1,n_days):
        yest = today - 1
        tot = S[yest] + I[yest] + R[yest] + T[yest] + D[yest]
        if print_daily:
            print(f"day={yest}, S={S[yest]}, I={I[yest]}, R={R[yest]}, T={T[yest]}, D={D[yest]}, tot={tot}")
        S.append(S[yest] - beta_daily_inf_rate * S[yest] * (I[yest] / n_pop))
        I.append(I[yest] + beta_daily_inf_rate * S[yest] * (I[yest] / n_pop) - 
                   (1 / mean_days_infectuous) * I[yest])
        R.append(R[yest] + (1 / mean_days_infectuous) * (I[yest] * (1 - death_prob)))
        T.append(T[yest] + (1 / mean_days_infectuous) * (I[yest] * death_prob) -
                    (1 / mean_days_to_death) * T[yest])
        D.append(D[yest] + (1 / mean_days_to_death) * T[yest])
        
    
    df = pd.DataFrame({'day':list(range(1, n_days + 1)), 
                    'S': S, 'I':I, 'R': R, 'T': T, 'D': D})
    df_long = df.melt(id_vars=['day'], 
                      value_vars=['S', 'I', 'R', 'T', 'D'],
                     var_name='state',
                     value_name='count')
    label_day = n_days - 1
    df_label = pd.DataFrame({'day':[n_days]*5, 
                             'count':[S[label_day], I[label_day], R[label_day], T[label_day], D[label_day]],
                             'label':['S', 'I', 'R', 'T', f'D:{round(D[n_days - 1])}']})
    #print(df_long)
    print((
        ggplot(df_long, aes(x='day', y='count')) +
        geom_point()) +
        geom_label(data = df_label, mapping=aes(label='label'))
        )

interactive(children=(IntSlider(value=200000000, description='n_pop', max=600000000, min=-200000000), IntSlide…

IndexError: list assignment index out of range

[1, 2, 3, 4, 5, 6, 7, 8, 9]
